# M3_AI2: Web Scrapping de películas
#### Autor: Leandro Gutierrez
#### Este documento intenta dar respuesta a la actividad individual 2 propuesta en el Modulo **Gestión de Datos y Datos Digitales** del **Master en Big Data y Ciencia de Datos**. En él se describirán cada uno de los enunciados postulados y los resultados obtenidos a través del uso de Python y Jupyter Notebook.
#### Julio 13, 2024

## Enunciado
El objetivo de esta actividad es obtener información sobre las películas a partir de la base de datos de IMDB. En concreto se pide mostrar los siguientes datos de las primeras 50 películas de del género Comedia en español:

- Nombre: Nombre de la película
- Rating: Puntuación media de la película en IMDB (si la tiene)
-Votos: Número de votos

A partir de la siguiente URL donde se lista las comedias (comedy) en español (spanish): https://www.imdb.com/search/title/?genres=comedy&languages=es contesta las siguientes preguntas:

1. Mediante el inspector de código del navegador, obtén y anota el selector CSS de los siguientes datos de la primera película:
   1. Nombre (tipo: texto)
   2. Puntuación del Rating (estrellas) de IMDB (tipo: numérico)
   3. Número de votos (votes) (tipo: numérico)
2. Identifica la parte común a los 3 selectores CSS del apartado 1 y la parte específica (última parte no común del selector CSS).
3. Construye la estructura de árbol de nodos sobre la página de búsqueda proporcionada al inicio del ejercicio (utiliza la siguiente cabecera):
        from urllib.request import Request, urlopen

        from lxml import html

        HEADERS={'User-Agent' : 'Magic Browser', 'Accept-Language': 'es-ES'}

        req =Request(url, headers=HEADERS)

        source = urlopen(req)

        tree = html.document_fromstring(str(source.read(), 'utf-8'))
        
4. Busca el selector CSS selectorCSS de la clase (class) de lista de películas y crea una lista de los resultados de las películas, a partir de este selector CSS:  

        resultado = tree.cssselect(selectorCSS)

        
- ¿Cuántos elementos tiene tu lista? :  len(resultado)
5. Construye un bucle que recorra la lista: resultado y que en cada elemento: resultado_i de la lista, realice 3 búsquedas: busquea_1, búsqueda_2 y búsqueda_3, una para cada selector CSS, utilizando sólo la parte específica del selector del apartado 2.
6. Para cada búsqueda realizada: resultado_1, resultado_2 y resultado_3, revisa si la nueva lista obtenida tiene longitud mayor que cero y en caso positivo, selecciona el primer elemento de la lista y haz un print de su contenido de texto: busquea_1[0].text_content()
7. Guarda los resultados obtenidos de forma iterativa en un data frame que tenga como resultado 25 filas y las 3 columnas con los datos solicitados en el apartado 1.

## Solución

In [76]:
# instalamos dependencias para entorno Colab
if 'google.colab' in str(get_ipython()):
    !pip install cssselect

### Obtención de los datos con **requests**

Para poder obtener el listado de Películas primero creamos la URL donde buscaremos los datos, además es necesario mencionar que el servidor como medida restrictiva solo contesta solicitudes gestionadas desde un navegador web, por lo que debemos agregar a nuestro request los headers necesarios para saltar esta restricción.

In [77]:
from lxml import html
import requests
import pandas as pd
import re

url = 'https://www.imdb.com/search/title/?genres=comedy&languages=es'
headers = {'content-type': 'application/json', 
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}

source = requests.get(url, headers=headers)

tree = html.document_fromstring(source.content)

Utilizando el inspector de código de nuestro navegador encontramos el selector `.ipc-metadata-list-summary-item` para recorrer cada uno de los elementos `<li>` de nuestra lista `<ul>`. Una vez identificado nuestro elemento fila, procedemos a encontrar los selectores que nos ayudaran a definir nuestros campos: **titulo**, **puntaje** y **votos**.

- Selector de **titulo**: `.ipc-metadata-list-summary-item .ipc-title__text`
- Selector de **puntaje**: `.ipc-metadata-list-summary-item .ipc-rating-star--imdb`
- Selector de **votos**: `.ipc-metadata-list-summary-item .ipc-rating-star--voteCount`

Siendo común a los tres selectores la sección correspondiente a la clase `ipc-metadata-list-summary-item` la cual adquiere cada elemento `<li>` de nuestra lista `<ul>`. Por lo tanto para seleccionar todos los elementos de nuestra lista hacemos:

In [78]:
peliculas_li = tree.cssselect('.ipc-metadata-list-summary-item')

print(f'Longitud de la lista: {len(peliculas_li)} elementos')


Longitud de la lista: 25 elementos


A continución dejo un bloque de código que quedó obsoleto, producto de una primera intención de resolución, la cual se determinó erronea por no poder asegurar coincidencia entre los elementos aislados de las 3 listas resultantes, es decir no podriamos asegurar a qué película le correspondería cada puntaje o cada cantidad de votos. 

In [79]:
# titulo_css = tree.cssselect(".ipc-metadata-list-summary-item .ipc-title__text")
# ranking_css = tree.cssselect(".ipc-metadata-list-summary-item .ipc-rating-star--imdb")
# votos_css = tree.cssselect(".ipc-metadata-list-summary-item .ipc-rating-star--voteCount")

# titulo = list(map(lambda x:x.text_content(),titulo_css))
# ranking = list(map(lambda x:x.text_content(),ranking_css))
# votos = list(map(lambda x:x.text_content(),votos_css))

La resolucíon adecuada del ejercicio se realiza con un `iterator` y un `bucle` para recorrerlo (en este caso utilizaremos la función `map`) y así poder asegurar correspondencia entre cada elemento de nuestra lista.

Además es necesario definir las expresiones regulares que nos permitirán parsear correctamente los textos adquiridos desde la fuente.

In [80]:
selector_titulo = '.ipc-title__text'
selector_puntaje = '.ipc-rating-star--imdb'
selector_votos = '.ipc-rating-star--voteCount'

patron_titulo = re.compile('[(\\d*).]\\s(.*)')
patron_puntaje = re.compile('(\\d+.\\d+|\\d+)\\s[(.*)]')
patron_votos = re.compile('(\\d+.\\d+|\\d+)([MK]?)')

def parsear(patron, base):
    if  len(re.findall(patron, base)) > 0:
        return re.findall(patron, base)[0]
    else:
        return None

def crear_obj(x):
    titulo = None
    if len(x.cssselect(selector_titulo)) > 0:
        titulo = str(parsear(patron_titulo, x.cssselect(selector_titulo)[0].text_content()))

    puntaje = None
    if len(x.cssselect(selector_puntaje)) > 0:
        puntaje_str = parsear(patron_puntaje, x.cssselect(selector_puntaje)[0].text_content())
        if puntaje_str != None:
            puntaje = float(puntaje_str)

    votos = None
    if len(x.cssselect(selector_votos)) > 0:
        # asumiremos que todos los votos estan expresados en miles de votos (K) o millones de votos (M)
        aux = parsear(patron_votos, x.cssselect(selector_votos)[0].text_content())
        
        if aux != None:
            base = aux[0]

            multiplicador = aux[1]

            if multiplicador == 'K':
                votos = float(base) * 1000
            elif multiplicador == 'M':
                votos = float(base) * 1000000
            else:
                votos = float(base)

    return {"titulo": titulo, "puntaje": puntaje, "votos": votos}
    
peliculas = map(crear_obj, peliculas_li)

ranking = pd.DataFrame(peliculas)

ranking.info()

ranking

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   titulo   25 non-null     object 
 1   puntaje  25 non-null     float64
 2   votos    25 non-null     float64
dtypes: float64(2), object(1)
memory usage: 732.0+ bytes


,titulo,puntaje,votos
0,Superdetective en Hollywood: Axel F.,6.5,43000.0
1,The Bear,8.6,228000.0
2,The Office,9.0,725000.0
3,Modern Family,8.5,491000.0
4,Cobra Kai,8.5,206000.0
5,Friends,8.9,1100000.0
6,Babylon,7.1,172000.0
7,Érase una vez en... Hollywood,7.6,860000.0
8,Seinfeld,8.9,355000.0
9,Crimen en el paraíso,7.8,29000.0


In [81]:
print(ranking.isna().sum())
print(ranking.isnull().sum())

titulo     0
puntaje    0
votos      0
dtype: int64
titulo     0
puntaje    0
votos      0
dtype: int64


Podemos observar que contamos con el listado de las primeras 25 peliculas o séries bajo los filtros "Comedia" (comedy) y "Español" (es). Así mismo, podemos notar que nuestro dataframe tiene 3 columnas **titulo, puntaje y votos**, las cuales son de tipo object, float64 y float64 respectivamente. Recordemos que el tipo de dato genérico para pandas es `object` el cual puede representar cualquier tipo valor, generamente siendo estos `strings`. Este mapeo surje a pesar de forzar el tipo de nuestra variable auxiliar `titulo` a string mediante el metodo `str()`. Podríamos intentar un cambio de tipo de columna una vez creado nuestro dataframe, se interpreta que esa función además de trivial no aporta valor al ejercicio.

También es necesario mencionar que la cantidad de votos se obtienen de multiplicar **la base** con su **multiplicador**, ambos obtenidos mediante la seleccion css de nuestro arbol y el parseo con la expresión regular definida para el campo votos. Consideramos que solo existen los multiplicadores `K` para referirse a Miles de votos y `M` para referirse a Millones de votos; y en su defecto el multiplicador será la unidad.